In [113]:

import pandas as pd
import importlib
import math

In [3]:
## TODO: write tests for roster scores

In [32]:
SAMPLE_PLAYERS = pd.DataFrame({
    'Player': ['antonio baldwin', 'corey dillon', 'elmer fudd', 'gregory halas', 'isaac jake','kyle lewis','mike north'],
    'player_id': [1,2,3,4,5,6,7],
    'position': ['QB','QB','QB','RB','RB','RB','RB'],
    'age' : [28,29,30,31,32,33,34]
    })

In [5]:
SAMPLE_ROSTER = pd.DataFrame({
    'owner': ['test_owner'],
    'players': [[1,2,3,4,5,6,7]]
    })

In [6]:
SAMPLE_PROJECTIONS = pd.DataFrame({
    'Player': ['antonio baldwin', 'corey dillon', 'elmer fudd', 'gregory halas', 'isaac jake','kyle lewis','mike north'],
    'FPTS': [350, 375, 400, 200, 225, 250, 275]
})

In [7]:
SAMPLE_KTC = pd.DataFrame({
    'Player': ['antonio baldwin', 'corey dillon', 'elmer fudd', 'gregory halas', 'isaac jake','kyle lewis','mike north'],
    'KTC_Value': [1000, 2000, 3000, 4000, 5000, 6000, 7000]
})

In [8]:
SAMPLE_F_CALC = pd.DataFrame({
    'Player': ['antonio baldwin', 'corey dillon', 'elmer fudd', 'gregory halas', 'isaac jake','kyle lewis','mike north'],
    'FantasyCalc_Value': [1500, 2500, 3500, 4500, 5500, 6500, 7500],
    'redraftValue': [3000, 6000, 500, 4500, 5500, 6500, 7500],
    'redraftDynastyValueDifference': [-1500, 3500, -3000, 0, 0, 0, 0]
})

In [214]:
imported_module = importlib.import_module("teams")
importlib.reload(imported_module)
from teams import *

league = build_league(SAMPLE_ROSTER,SAMPLE_PLAYERS,SAMPLE_PROJECTIONS,SAMPLE_KTC,SAMPLE_F_CALC)
for team in league:
   set_team_starters(team)

In [216]:
qbs = team.get_position_starters(POSITIONS.QB)
assert qbs.iloc[0]['Player'] == 'elmer fudd', "Elmer Fudd is the highest projected QB"
backup_qbs = team.get_position_backups(POSITIONS.QB)
assert backup_qbs.iloc[0]['Player'] == 'antonio baldwin', "Antonio Baldwin is the lowest projected QB"

In [217]:
new_team = copy_team(team)
assert team.roster_starters['Starter'].value_counts()[True] == 5
assert new_team.roster_starters['Starter'].value_counts()[False] == 7

In [218]:
imported_module = importlib.import_module("leaguestats")
importlib.reload(imported_module)
from leaguestats import *

In [219]:
def team_factory(initial, offset=0):
    team = Team(f'{offset}')
    if offset:
        team.roster_starters = pd.DataFrame({
            'position': ['QB','WR','WR','RB','TE','WR','RB','QB','QB','WR','WR','RB','TE'],
            'FPPG': [initial - i * offset for i in range(13)],
            'Starter': False
        })
        set_team_starters(team)
    return team

In [220]:
team1 = team_factory(200, 5)
team2 = team_factory(200, 10)
team3 = team_factory(200, 20)
testLeague = [team1, team2, team3]

testStats = LeagueStats()
calculate_league_starter_stats(testLeague, testStats)
calculate_league_bench_stats(testLeague, testStats)

league_wide_statistics = {
    'average_starter_ppg': 0,
    'median_starter_ppg': 0,
    'average_starters': {},
    'median_starters': {},
    'average_best_bench': {},
    'median_best_bench': {}
}
calculate_league_starter_stats_depr(testLeague, league_wide_statistics)
calculate_league_bench_stats_depr(testLeague, league_wide_statistics)

In [221]:
assert testStats.starter_ppg(average=True) == (team1.roster_starters.query('Starter')['FPPG'].sum() + team2.roster_starters.query('Starter')['FPPG'].sum() + team3.roster_starters.query('Starter')['FPPG'].sum()) / 3.0

assert testStats.position_ppg(POSITIONS.QB, starter=True, average=True) == [200.0]
start_wr_ppg = testStats.position_ppg(POSITIONS.WR, starter=True, average=True)
assert math.isclose(start_wr_ppg[0], (195 + 190 + 180) / 3.0, abs_tol=0.1)
assert math.isclose(start_wr_ppg[1], (190 + 180 + 160) / 3.0, abs_tol=0.1)

assert math.isclose(testStats.position_ppg(POSITIONS.SFLEX, bench=True, average=True)[0], (160 + 120 + 40) / 3.0, abs_tol=0.1)
bench_flex_ppg = testStats.position_ppg(POSITIONS.FLEX, bench=True, average=True)
assert math.isclose(bench_flex_ppg[0], (155 + 110 + 20) / 3.0, abs_tol=0.1)
assert math.isclose(bench_flex_ppg[1], (150 + 100 + 0) / 3.0, abs_tol=0.1)
assert math.isclose(bench_flex_ppg[2], (145 + 90 - 20) / 3.0, abs_tol=0.1)

In [245]:
simple_start_qb_fppg = 100
simple_start_wr_fppg = 80
simple_team = Team('simple')
simple_team.roster_starters = pd.DataFrame({
            'position': ['QB','WR','QB','WR','WR'],
            'FPPG': [simple_start_qb_fppg, simple_start_wr_fppg, 60, 40, 20],
            'Starter': [True, True, False, False, False]
        })
simple_team.starting_roster['QB'] = simple_team.roster_starters.loc[0:0].copy()
simple_team.starting_roster['WR'] = simple_team.roster_starters.loc[1:1].copy()


simple_statistics = {
    'average_starter_ppg': 180,
    'median_starter_ppg': 200,
    'average_starters' : {
        'QB': [simple_start_qb_fppg * 2],
        'WR': [100]
    },
    'median_starters' : {
        'QB': [simple_start_qb_fppg * 2],
        'WR': [200]
    },
    'average_best_bench' : {
        'QB': [100],
        'WR': [100, 100]
    },
    'median_best_bench' : {
        'QB': [200],
        'WR': [100, 100]
    }
}
calculate_overall_starter_score_for_team(simple_team, simple_statistics)
calculate_score_for_position(simple_team, simple_statistics, POSITIONS.QB, True, 1)
calculate_score_for_position(simple_team, simple_statistics, POSITIONS.WR, True, 1)
calculate_score_for_position(simple_team, simple_statistics, POSITIONS.QB, False, 1)
calculate_score_for_position(simple_team, simple_statistics, POSITIONS.WR, False, 2)

In [249]:
assert math.isclose(simple_team.combined_starter_score(), 95)

assert math.isclose(simple_team.position_starter_score(POSITIONS.QB), 50)
assert math.isclose(simple_team.position_starter_score(POSITIONS.WR), 60)
assert math.isclose(simple_team.position_starter_score(POSITIONS.WR), 60)

assert math.isclose(simple_team.position_bench_score(POSITIONS.QB), 45)
assert math.isclose(simple_team.position_bench_score(POSITIONS.WR), 33 + 1/3)